In [1]:
import os
import xarray as xr
import geopandas as gpd  # type: ignore

from dotenv import load_dotenv
import sqlalchemy as sq
import sys
import pandas as pd
import matplotlib.pyplot as plt  # type: ignore

sys.path.append("../")
from Shared.DataService import DataService

/home/student/anghandk/.local/lib/python3.10/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_4041198/3748365637.py:3: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using Py

In [49]:
SOIL_GEOM_TABLE = "soil_geometry"
SOIL_COMP_TABLE = "soil_components"
SOIL_SURRONDINGS_TABLE = "soil_surronding_land"
SOIL_DATA_TABLE = "soil_data"

load_dotenv()
PG_DB = os.getenv("POSTGRES_DB")
PG_ADDR = os.getenv("POSTGRES_ADDR")
PG_PORT = os.getenv("POSTGRES_PORT")
PG_USER = os.getenv("POSTGRES_USER")
PG_PW = os.getenv("POSTGRES_PW")

In [77]:
if (
    PG_DB is None
    or PG_ADDR is None
    or PG_PORT is None
    or PG_USER is None
    or PG_PW is None
):
    raise ValueError("Environment variables not set")
db = DataService(PG_DB, PG_ADDR, int(PG_PORT), PG_USER, PG_PW)
conn = db.connect()

In [78]:
query = sq.text("select * FROM public.soil_data")
sm_df = pd.read_sql(query, conn)

In [79]:
sm_df

,id,province,code,modifier,name,kind,water_table,root_restrict,restr_type,drainage,...,ksat,water_reten_0,water_reten_10,water_reten_33,water_reten_1500,bulk_density,elec_cond,calc_equiv,decomp_class,percnt_wood
0,MB$AR~~~~~N,MB,$AR,~~~~~,Acidic Bedrock,N,-,-,-,-,...,10.00,85,70,50,15,0.15,0,0,-9,0
1,MB$AR~~~~~N,MB,$AR,~~~~~,Acidic Bedrock,N,-,-,-,-,...,-9.00,-9,-9,-9,-9,2.80,-9,-9,-9,-9
2,MB$CB~~~~~N,MB,$CB,~~~~~,Cobble Beach,M,YU,0,-,I,...,60.00,34,6,5,1,1.80,0,20,-9,-9
3,MB$CB~~~~~N,MB,$CB,~~~~~,Cobble Beach,M,YU,0,-,I,...,50.00,37,6,5,1,1.70,0,20,-9,-9
4,MB$CB~~~~~N,MB,$CB,~~~~~,Cobble Beach,M,YU,0,-,I,...,25.00,40,6,5,1,1.60,0,15,-9,-9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36050,SKYKHS~~~~A,SK,YKH,S~~~~,YORKTON GLCA.BLC,M,YU,3,SA,I,...,3.76,47,-9,31,17,1.40,6,10,-9,-9
36051,SKYKHS~~~~A,SK,YKH,S~~~~,YORKTON GLCA.BLC,M,YU,3,SA,I,...,6.21,43,-9,28,16,1.50,6,21,-9,-9
36052,SKYKH~~~~~A,SK,YKH,~~~~~,YORKTON GLCA.BLC,M,YU,0,-,I,...,5.36,55,-9,33,20,1.20,1,5,-9,-9
36053,SKYKH~~~~~A,SK,YKH,~~~~~,YORKTON GLCA.BLC,M,YU,0,-,I,...,3.76,47,-9,31,17,1.40,1,10,-9,-9


In [80]:
sm_df = (
    sm_df.groupby(["id","code", "province"]).agg({"name":["max"]}).reset_index()
    )

In [81]:
sm_df

,id,code,province,name
,,,,max
0,ABABCgl~~~N,ABC,AB,ATHABASCA
1,ABABCst~~~A,ABC,AB,ATHABASCA
2,ABABCst~~~N,ABC,AB,ATHABASCA
3,ABABC~~~~~A,ABC,AB,ATHABASCA
4,ABABC~~~~~N,ABC,AB,ATHABASCA
...,...,...,...,...
8353,SKYKH43~~~A,YKH,SK,YORKTON GLCA.BLC
8354,SKYKHS26~~A,YKH,SK,YORKTON GLCA.BLC
8355,SKYKHS43~~A,YKH,SK,YORKTON GLCA.BLC


In [ ]:
TABLE = "agg_soil_data"

In [51]:
query = sq.text("select * FROM public.soil_data")
soil_data = pd.read_sql(query, conn)
soil_data.head()

,id,province,code,modifier,name,kind,water_table,root_restrict,restr_type,drainage,...,ksat,water_reten_0,water_reten_10,water_reten_33,water_reten_1500,bulk_density,elec_cond,calc_equiv,decomp_class,percnt_wood
0,MB$AR~~~~~N,MB,$AR,~~~~~,Acidic Bedrock,N,-,-,-,-,...,10.0,85,70,50,15,0.15,0,0,-9,0
1,MB$AR~~~~~N,MB,$AR,~~~~~,Acidic Bedrock,N,-,-,-,-,...,-9.0,-9,-9,-9,-9,2.80,-9,-9,-9,-9
2,MB$CB~~~~~N,MB,$CB,~~~~~,Cobble Beach,M,YU,0,-,I,...,60.0,34,6,5,1,1.80,0,20,-9,-9
3,MB$CB~~~~~N,MB,$CB,~~~~~,Cobble Beach,M,YU,0,-,I,...,50.0,37,6,5,1,1.70,0,20,-9,-9
4,MB$CB~~~~~N,MB,$CB,~~~~~,Cobble Beach,M,YU,0,-,I,...,25.0,40,6,5,1,1.60,0,15,-9,-9


In [54]:
soil_data = soil_data[["id", "province", "kind", "water_table", "root_restrict", "drainage", "parent_material_texture_1", "parent_material_chemical_1", "mode_of_depo_1", 'percnt_coarse_frag', 'sand_texture', 'total_sand', 'total_silt', 'total_clay', 'percnt_carbon', 'calcium_ph', 'proj_ph',  'water_reten_0', 'water_reten_10', 'water_reten_33',
       'water_reten_1500', 'bulk_density','elec_cond']]

In [73]:
soil_data

,soil_id,province,kind,water_table,root_restrict,drainage,parent_material_texture_1,parent_material_chemical_1,mode_of_depo_1,percnt_coarse_frag,...,total_clay,percnt_carbon,calcium_ph,proj_ph,water_reten_0,water_reten_10,water_reten_33,water_reten_1500,bulk_density,elec_cond
0,~~~~~N,MB,N,-,-,-,-,-,RKIA,0,...,-9,34.8,4.9,5.4,85,70,50,15,0.15,0
1,~~~~~N,MB,N,-,-,-,-,-,RKIA,-9,...,-9,-9.0,-9.0,-9.0,-9,-9,-9,-9,2.80,-9
2,~~~~~N,MB,M,YU,0,I,FR,UD,LACU,75,...,0,0.1,7.8,8.2,34,6,5,1,1.80,0
3,~~~~~N,MB,M,YU,0,I,FR,UD,LACU,35,...,0,0.1,7.8,8.2,37,6,5,1,1.70,0
4,~~~~~N,MB,M,YU,0,I,FR,UD,LACU,25,...,2,0.1,7.8,8.2,40,6,5,1,1.60,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36050,S~~~~A,SK,M,YU,3,I,M,SA,TILL,0,...,23,1.1,7.7,7.8,47,-9,31,17,1.40,6
36051,S~~~~A,SK,M,YU,3,I,M,SA,TILL,0,...,20,0.3,7.9,8.0,43,-9,28,16,1.50,6
36052,~~~~~A,SK,M,YU,0,I,M,VC,TILL,0,...,20,4.3,7.4,7.5,55,-9,33,20,1.20,1
36053,~~~~~A,SK,M,YU,0,I,M,VC,TILL,0,...,23,1.1,7.7,7.8,47,-9,31,17,1.40,1


In [15]:
query = sq.text("select * FROM public.soil_surronding_land")
surronding_soil = pd.read_sql(query, conn)
surronding_soil.head()

,poly_id,land_area,water_area,fresh_area,ocean_area,total_area
0,1001,844951,10923,718,10205,855874
1,1002,111030,0,0,0,111030
2,1003,46707,0,0,0,46707
3,1004,123419,0,0,0,123419
4,1005,185880,35167,389,34778,221047


In [42]:
surronding_soil = surronding_soil[["poly_id", "land_area"]]

In [43]:
surronding_soil

,poly_id,land_area
0,1001,844951
1,1002,111030
2,1003,46707
3,1004,123419
4,1005,185880
...,...,...
12338,9687011,60471
12339,9787001,25619
12340,9787002,7291
12341,9787003,108903


In [16]:
query = sq.text("select * FROM public.soil_components")
soil_components = pd.read_sql(query, conn)
soil_components.head()

,poly_id,cmp,percent,slope,stone,surface_area,province,soil_code,modifier,profile,soil_id,coarse_frag_1,coarse_frag_2,depth,water_holding_cap
0,242021,1,40,A,U,D,AB,BUF,gl~~~,N,ABBUFgl~~~N,A,A,5,4
1,242021,2,30,A,N,U,AB,KEG,~~~~~,N,ABKEG~~~~~N,A,A,5,5
2,242021,3,30,A,N,F13,AB,MLD,~~~~~,N,ABMLD~~~~~N,A,A,5,5
3,242022,1,40,A,N,B07,AB,MKW,~~~~~,N,ABMKW~~~~~N,-,-,4,5
4,242022,2,30,A,N,F13,AB,MLD,~~~~~,N,ABMLD~~~~~N,A,A,5,5


In [44]:
soil_components = soil_components[["poly_id", "cmp", "percent", "province", "profile", "soil_id", "water_holding_cap"]]

In [76]:
soil_components

0        ABBUFgl~~~N
1        ABKEG~~~~~N
2        ABMLD~~~~~N
3        ABMKW~~~~~N
4        ABMLD~~~~~N
            ...     
20416    ABZGW~~~~~N
20417    ABMAB~~~~~A
20418    ABROL~~~~~A
20419    ABHDY~~~~~A
20420    ABZGW~~~~~N
Name: soil_id, Length: 20421, dtype: object

In [17]:
query = sq.text("select * FROM public.soil_geometry")
soil_geometry = pd.read_sql(query, conn)
soil_geometry.head()

,area,perimeter,poly_id,geometry
0,0.703365,12.299814,20002,0103000020130D0000010000009D00000094649194BB95...
1,1.448621,11.987536,20008,0103000020130D0000010000004F0000009E3180F0D918...
2,1.836356,19.000614,2002,0103000020130D000001000000790100009F4B3A656210...
3,0.801305,9.671116,20006,0103000020130D000001000000010100005AA7A6A97972...
4,0.515888,8.755135,16001,0103000020130D000001000000B4000000C2B7907EE88B...


In [56]:
db.cleanup()

In [57]:
soil_data.rename(columns={"id":"soil_id"}, inplace=True)

/tmp/ipykernel_4041198/220420652.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  soil_data.rename(columns={"id":"soil_id"}, inplace=True)


In [58]:
soil_data

,soil_id,province,kind,water_table,root_restrict,drainage,parent_material_texture_1,parent_material_chemical_1,mode_of_depo_1,percnt_coarse_frag,...,total_clay,percnt_carbon,calcium_ph,proj_ph,water_reten_0,water_reten_10,water_reten_33,water_reten_1500,bulk_density,elec_cond
0,MB$AR~~~~~N,MB,N,-,-,-,-,-,RKIA,0,...,-9,34.8,4.9,5.4,85,70,50,15,0.15,0
1,MB$AR~~~~~N,MB,N,-,-,-,-,-,RKIA,-9,...,-9,-9.0,-9.0,-9.0,-9,-9,-9,-9,2.80,-9
2,MB$CB~~~~~N,MB,M,YU,0,I,FR,UD,LACU,75,...,0,0.1,7.8,8.2,34,6,5,1,1.80,0
3,MB$CB~~~~~N,MB,M,YU,0,I,FR,UD,LACU,35,...,0,0.1,7.8,8.2,37,6,5,1,1.70,0
4,MB$CB~~~~~N,MB,M,YU,0,I,FR,UD,LACU,25,...,2,0.1,7.8,8.2,40,6,5,1,1.60,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36050,SKYKHS~~~~A,SK,M,YU,3,I,M,SA,TILL,0,...,23,1.1,7.7,7.8,47,-9,31,17,1.40,6
36051,SKYKHS~~~~A,SK,M,YU,3,I,M,SA,TILL,0,...,20,0.3,7.9,8.0,43,-9,28,16,1.50,6
36052,SKYKH~~~~~A,SK,M,YU,0,I,M,VC,TILL,0,...,20,4.3,7.4,7.5,55,-9,33,20,1.20,1
36053,SKYKH~~~~~A,SK,M,YU,0,I,M,VC,TILL,0,...,23,1.1,7.7,7.8,47,-9,31,17,1.40,1


In [65]:
merge_df = soil_components.merge(soil_data, on=["soil_id"], how="inner")

In [68]:
merge_df = merge_df.merge(soil_geometry, on="poly_id", how="inner")

In [69]:
merge_df

,poly_id,cmp,percent,province_x,profile,soil_id,water_holding_cap,province_y,kind,water_table,...,proj_ph,water_reten_0,water_reten_10,water_reten_33,water_reten_1500,bulk_density,elec_cond,area,perimeter,geometry
0,242021,1,40,AB,N,ABBUFgl~~~N,4,AB,M,YU,...,4.6,85,65,-9,20,0.10,0,0.188748,4.718521,0103000020130D000001000000F6000000B2E81831B98B...
1,242021,1,40,AB,N,ABBUFgl~~~N,4,AB,M,YU,...,4.8,51,-9,22,10,1.30,0,0.188748,4.718521,0103000020130D000001000000F6000000B2E81831B98B...
2,242021,1,40,AB,N,ABBUFgl~~~N,4,AB,M,YU,...,5.0,42,-9,32,18,1.55,0,0.188748,4.718521,0103000020130D000001000000F6000000B2E81831B98B...
3,242021,1,40,AB,N,ABBUFgl~~~N,4,AB,M,YU,...,6.5,42,-9,31,18,1.55,0,0.188748,4.718521,0103000020130D000001000000F6000000B2E81831B98B...
4,242021,1,40,AB,N,ABBUFgl~~~N,4,AB,M,YU,...,5.6,43,-9,28,16,1.50,0,0.188748,4.718521,0103000020130D000001000000F6000000B2E81831B98B...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40005,1016001,2,30,AB,N,ABHDX~~~~~N,4,AB,M,NO,...,7.6,47,-9,26,13,1.40,0,0.032778,1.781507,0103000020130D000001000000B00000004A5D764AA97A...
40006,1016001,2,30,AB,N,ABHDX~~~~~N,4,AB,M,NO,...,7.6,47,-9,41,22,1.40,0,0.032778,1.781507,0103000020130D000001000000B00000004A5D764AA97A...
40007,1016001,2,30,AB,N,ABHDX~~~~~N,4,AB,M,NO,...,7.7,47,-9,37,19,1.40,0,0.032778,1.781507,0103000020130D000001000000B00000004A5D764AA97A...
40008,1016001,2,30,AB,N,ABHDX~~~~~N,4,AB,M,NO,...,7.7,42,-9,15,8,1.55,0,0.032778,1.781507,0103000020130D000001000000B00000004A5D764AA97A...


In [70]:
merge_df = merge_df.merge(surronding_soil, on="poly_id", how="inner")

In [71]:
merge_df

,poly_id,cmp,percent,province_x,profile,soil_id,water_holding_cap,province_y,kind,water_table,...,water_reten_0,water_reten_10,water_reten_33,water_reten_1500,bulk_density,elec_cond,area,perimeter,geometry,land_area
0,242021,1,40,AB,N,ABBUFgl~~~N,4,AB,M,YU,...,85,65,-9,20,0.10,0,0.188748,4.718521,0103000020130D000001000000F6000000B2E81831B98B...,118437
1,242021,1,40,AB,N,ABBUFgl~~~N,4,AB,M,YU,...,51,-9,22,10,1.30,0,0.188748,4.718521,0103000020130D000001000000F6000000B2E81831B98B...,118437
2,242021,1,40,AB,N,ABBUFgl~~~N,4,AB,M,YU,...,42,-9,32,18,1.55,0,0.188748,4.718521,0103000020130D000001000000F6000000B2E81831B98B...,118437
3,242021,1,40,AB,N,ABBUFgl~~~N,4,AB,M,YU,...,42,-9,31,18,1.55,0,0.188748,4.718521,0103000020130D000001000000F6000000B2E81831B98B...,118437
4,242021,1,40,AB,N,ABBUFgl~~~N,4,AB,M,YU,...,43,-9,28,16,1.50,0,0.188748,4.718521,0103000020130D000001000000F6000000B2E81831B98B...,118437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40005,1016001,2,30,AB,N,ABHDX~~~~~N,4,AB,M,NO,...,47,-9,26,13,1.40,0,0.032778,1.781507,0103000020130D000001000000B00000004A5D764AA97A...,22950
40006,1016001,2,30,AB,N,ABHDX~~~~~N,4,AB,M,NO,...,47,-9,41,22,1.40,0,0.032778,1.781507,0103000020130D000001000000B00000004A5D764AA97A...,22950
40007,1016001,2,30,AB,N,ABHDX~~~~~N,4,AB,M,NO,...,47,-9,37,19,1.40,0,0.032778,1.781507,0103000020130D000001000000B00000004A5D764AA97A...,22950
40008,1016001,2,30,AB,N,ABHDX~~~~~N,4,AB,M,NO,...,42,-9,15,8,1.55,0,0.032778,1.781507,0103000020130D000001000000B00000004A5D764AA97A...,22950


In [83]:
merge_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40010 entries, 0 to 40009
Data columns (total 33 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   poly_id                     40010 non-null  int64  
 1   cmp                         40010 non-null  int64  
 2   percent                     40010 non-null  int64  
 3   province_x                  40010 non-null  object 
 4   profile                     40010 non-null  object 
 5   soil_id                     40010 non-null  object 
 6   water_holding_cap           40010 non-null  object 
 7   province_y                  40010 non-null  object 
 8   kind                        40010 non-null  object 
 9   water_table                 40010 non-null  object 
 10  root_restrict               40010 non-null  object 
 11  drainage                    39968 non-null  object 
 12  parent_material_texture_1   39969 non-null  object 
 13  parent_material_chemical_1  399

In [86]:
merge_df.describe()

,poly_id,cmp,percent,percnt_coarse_frag,total_sand,total_silt,total_clay,percnt_carbon,calcium_ph,proj_ph,water_reten_0,water_reten_10,water_reten_33,water_reten_1500,bulk_density,elec_cond,area,perimeter,land_area
count,4.001000e+04,40010.000000,40010.000000,40010.000000,40010.000000,40010.000000,40010.000000,40010.000000,40010.000000,40010.000000,40010.000000,40010.000000,40010.000000,40010.000000,40010.000000,40010.000000,40010.000000,40010.000000,40010.000000
mean,8.198509e+05,3.409898,20.753662,2.773482,34.559635,27.423294,20.628768,6.542347,6.482117,6.821100,52.208723,5.921870,27.596176,16.913747,1.236779,0.621745,0.078726,2.030269,55539.264584
std,1.027385e+06,2.314419,16.164786,8.208136,27.959611,20.049104,18.128097,14.412051,1.525362,1.445054,16.083768,25.973915,16.413605,6.897139,0.487454,1.817235,0.096619,1.546175,64304.369288
min,2.420210e+05,1.000000,1.000000,-9.000000,-9.000000,-9.000000,-9.000000,-9.000000,-9.000000,-9.000000,-9.000000,-9.000000,-9.000000,-9.000000,-9.000000,-9.000000,0.000250,0.063169,184.000000
25%,6.250110e+05,2.000000,10.000000,0.000000,15.000000,13.000000,9.000000,0.300000,5.700000,6.100000,43.000000,-9.000000,20.000000,13.000000,1.300000,0.000000,0.026846,1.038607,19586.000000
50%,7.240050e+05,3.000000,16.000000,0.000000,35.000000,32.000000,22.000000,0.900000,6.800000,7.000000,47.000000,-9.000000,31.000000,18.000000,1.400000,0.000000,0.048932,1.613416,36057.000000
75%,7.930080e+05,4.000000,30.000000,3.000000,48.000000,40.000000,30.000000,2.400000,7.500000,7.800000,51.000000,17.000000,37.000000,21.000000,1.500000,1.000000,0.092423,2.475323,65858.000000
max,9.787004e+06,17.000000,100.000000,80.000000,100.000000,83.000000,84.000000,60.000000,9.400000,9.700000,96.000000,75.000000,65.000000,42.000000,2.800000,22.000000,1.326780,12.210245,751088.000000
